In [13]:
import pandas as pd
import numpy as np
import pickle
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, LSTM, Embedding, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Reshape

# Tải các tài nguyên NLTK
nltk.download('stopwords')
nltk.download('wordnet')

# Bước 1: Tải dữ liệu
df = pd.read_csv('data/merged_data.csv')

# Bước 2: Mã hóa nhãn cảm xúc thành số
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# Bước 3: Xóa các dòng có giá trị NaN trong cột 'content'
df.dropna(subset=['content'], inplace=True)
X = df['content']
y = df['label_encoded']

# Bước 4: Phân chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Bước 5: Tiền xử lý văn bản
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if isinstance(text, str):
        text = text.lower()
        text = re.sub(r'[^a-zA-ZÀ-ỹ\s]', '', text)
        tokens = text.split()
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
        return ' '.join(tokens)
    else:
        return ''

# Tiến hành tiền xử lý dữ liệu
X_train = X_train.apply(preprocess_text)
X_test = X_test.apply(preprocess_text)

# Bước 6: Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# Chuyển đổi văn bản thành chuỗi số
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding để các chuỗi có cùng chiều dài
maxlen = max([len(seq) for seq in X_train_seq])
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen, padding='post')

# Chuyển đổi nhãn thành dạng one-hot
y_train = to_categorical(y_train, num_classes=len(label_encoder.classes_))
y_test = to_categorical(y_test, num_classes=len(label_encoder.classes_))

# Thiết lập early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Xây dựng mô hình kết hợp CNN và LSTM
model_combined = Sequential()
model_combined.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=maxlen))
model_combined.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model_combined.add(GlobalMaxPooling1D())
model_combined.add(Reshape((1, 128)))  # Thay đổi kích thước cho LSTM
model_combined.add(LSTM(128, return_sequences=False))
model_combined.add(Dense(len(label_encoder.classes_), activation='softmax'))
model_combined.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình kết hợp
model_combined.fit(X_train_pad, y_train, epochs=10, batch_size=10, validation_data=(X_test_pad, y_test), callbacks=[early_stopping])

# Đánh giá mô hình
cnn_loss, cnn_acc = model_combined.evaluate(X_test_pad, y_test)
print(f'Combined Model Loss: {cnn_loss}, Accuracy: {cnn_acc}')

# Đánh giá chi tiết
y_pred = model_combined.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_test.argmax(axis=1), y_pred_classes, target_names=label_encoder.classes_))

# Lưu mô hình
model_combined.save('sentiment_model_combined.h5')

# Lưu Tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Lưu Label Encoder
with open('label_encoder.pickle', 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Epoch 1/10


c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


5367/5367 ━━━━━━━━━━━━━━━━━━━━ 206s 38ms/step - accuracy: 0.8349 - loss: 0.4141 - val_accuracy: 0.8792 - val_loss: 0.3002
Epoch 2/10
5367/5367 ━━━━━━━━━━━━━━━━━━━━ 170s 32ms/step - accuracy: 0.9096 - loss: 0.2427 - val_accuracy: 0.8803 - val_loss: 0.2989
Epoch 3/10
5367/5367 ━━━━━━━━━━━━━━━━━━━━ 175s 33ms/step - accuracy: 0.9299 - loss: 0.1890 - val_accuracy: 0.8813 - val_loss: 0.3239
Epoch 4/10
5367/5367 ━━━━━━━━━━━━━━━━━━━━ 174s 32ms/step - accuracy: 0.9520 - loss: 0.1368 - val_accuracy: 0.8753 - val_loss: 0.3614
Epoch 5/10
5367/5367 ━━━━━━━━━━━━━━━━━━━━ 174s 32ms/step - accuracy: 0.9616 - loss: 0.1103 - val_accuracy: 0.8734 - val_loss: 0.4824
420/420 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.8819 - loss: 0.2937
Combined Model Loss: 0.2989165186882019, Accuracy: 0.8803011178970337
420/420 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step
              precision    recall  f1-score   support

         NEG       0.92      0.89      0.90      4493
         NEU       0.87      0.83      0.85      

In [85]:
import pandas as pd
import csv  # Import thư viện để kiểm soát việc trích dẫn ký tự

# Đọc dữ liệu từ file CSV với encoding phù hợp
data = pd.read_csv('data/comments_data_ncds.csv', encoding='utf-8')

# Chuyển thành DataFrame và chỉ lấy các cột cần thiết
df = data[["content", "rating"]].copy()

# Gán nhãn
df["label"] = df["rating"].apply(lambda x: "POS" if x >= 4 else ("NEU" if x == 3 else "NEG"))


# Đổi tên cột rating thành start
df.rename(columns={"rating": "start"}, inplace=True)

# Xóa dấu " trong nội dung một cách an toàn
df["content"] = df["content"].str.replace(r"[^a-zA-ZÀ-ỹ ]", "", regex=True)

# Loại bỏ dòng trống
df = df[df["content"].str.strip() != ""]

# Loại bỏ dòng trùng lặp
df = df.drop_duplicates()
df = df[["content", "label", "start"]]

# Xuất dữ liệu sạch mà không có dấu ngoặc kép
df.to_csv('data/cleaned_comments_data.csv', index=False, header=True, encoding='utf-8', quoting=csv.QUOTE_NONE, escapechar='\\')

print("Dữ liệu sau khi làm sạch và lưu file thành công!")


Dữ liệu sau khi làm sạch và lưu file thành công!


In [86]:
import pandas as pd

# Đọc hai file CSV, bỏ qua dòng tiêu đề nếu có
df1 = pd.read_csv('data/cleaned_comments_data.csv', header=0, names=["content", "label", "start"])
df2 = pd.read_csv('data/data2.csv', header=0, names=["content", "label", "start"])

# Ghép hai DataFrame lại
merged_df = pd.concat([df1, df2], ignore_index=True)

# Loại bỏ dòng chứa tiêu đề nếu bị đọc vào dữ liệu
merged_df = merged_df[merged_df["start"] != "start"]

# Loại bỏ dòng có giá trị 'start' không hợp lệ (không phải số)
merged_df = merged_df[pd.to_numeric(merged_df["start"], errors="coerce").notna()]

# Chuyển 'start' về số nguyên
merged_df["start"] = merged_df["start"].astype(float).astype(int)

# Lưu vào file mới với tiêu đề cột
merged_df.to_csv('data/merged_data.csv', index=False, header=True, encoding='utf-8')

print("Ghép file thành công và lưu vào merged_data.csv với tiêu đề cột! Cột 'start' đã chuyển về số nguyên.")


Ghép file thành công và lưu vào merged_data.csv với tiêu đề cột! Cột 'start' đã chuyển về số nguyên.


In [10]:
import pandas as pd

df1 = pd.read_csv('data/merged_data.csv', header=0, names=["content", "label", "start"])
print(df1['label'].value_counts())

label
POS    23437
NEG    22467
NEU    21203
Name: count, dtype: int64


In [7]:
import pandas as pd
import random
import os

# Mở rộng danh sách các cụm từ mang sắc thái trung tính
neutral_adjectives = ["bình thường", "ổn", "tạm được", "không tốt không xấu", "chấp nhận được", "trung bình", "khá ổn", "vừa phải", "thông thường", "đạt yêu cầu"]
neutral_nouns = ["sản phẩm", "dịch vụ", "chất lượng", "giao hàng", "đóng gói", "giá cả", "cửa hàng", "website", "thái độ", "trải nghiệm"]
neutral_verbs = ["là", "có", "khá", "cũng", "tương đối", "ở mức", "vẫn", "vừa", "thấy", "được"]

# Các mẫu câu trung tính
sentence_templates = [
    "{noun} {verb} {adjective}.",
    "{noun} này {verb} {adjective}.",
    "Tôi thấy {noun} {verb} {adjective}.",
    "{noun} {verb} {adjective} thôi."
]

# Tạo danh sách các câu đánh giá trung tính ngẫu nhiên
neutral_reviews = []
for _ in range(10000):
    adjective = random.choice(neutral_adjectives)
    noun = random.choice(neutral_nouns)
    verb = random.choice(neutral_verbs)
    template = random.choice(sentence_templates)
    review = template.format(adjective=adjective, noun=noun, verb=verb)
    neutral_reviews.append(review)

# Tạo danh sách các label
neu_labels = ["NEU"] * 10000

# Tạo danh sách các giá trị start (3 sao)
starts = [3] * 10000

# Tạo DataFrame
df_neutral = pd.DataFrame({"content": neutral_reviews, "label": neu_labels, "start": starts})

# Lưu DataFrame vào file CSV
output_dir = "data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_file = os.path.join(output_dir, "neutral_reviews.csv")
df_neutral.to_csv(output_file, index=False, encoding="utf-8")

print(f"Đã lưu dữ liệu trung tính vào file: {output_file}")

Đã lưu dữ liệu trung tính vào file: data\neutral_reviews.csv


In [8]:
import pandas as pd
import random
import os

# Mở rộng danh sách các cụm từ mang sắc thái tiêu cực
negative_adjectives = ["tệ hại", "kém chất lượng", "thất vọng", "dở tệ", "bực mình", "không đáng tiền", "hỏng hóc", "chậm chạp", "lừa đảo", "phiền phức", "tồi tệ", "ghê tởm", "kinh khủng", "rác rưởi", "bỏ đi", "kém", "dở", "xấu", "xoàng", "thảm hại"]
negative_nouns = ["sản phẩm", "dịch vụ", "chất lượng", "giao hàng", "đóng gói", "giá cả", "cửa hàng", "website", "thái độ", "trải nghiệm", "nhà sản xuất", "người bán", "chính sách", "hỗ trợ", "bảo hành"]
negative_verbs = ["là", "có", "gây ra", "khiến tôi", "làm tôi", "không tốt", "không như mong đợi", "không hài lòng", "không đáng mua", "cần phải cải thiện", "thật", "quá", "rất", "không thể chấp nhận", "không nên mua"]

# Các mẫu câu (đã bỏ .capitalize())
sentence_templates = [
    "{noun} {verb} {adjective}.",
    "{noun} này thật {adjective}",
]

# Các mẫu câu không cần capitalize
sentence_templates_no_cap = [
    "Tôi rất {adjective} về {noun}.",
    "Thật {adjective} khi {noun} lại {verb} như vậy.",
    "Tôi không hiểu sao {noun} lại có thể {verb} đến thế."
]

# Tạo danh sách các câu đánh giá tiêu cực ngẫu nhiên
negative_reviews = []
for _ in range(10000):
    adjective = random.choice(negative_adjectives)
    noun = random.choice(negative_nouns)
    verb = random.choice(negative_verbs)

    # Chọn ngẫu nhiên mẫu câu
    if random.choice([True, False]):
        template = random.choice(sentence_templates)
    else:
        template = random.choice(sentence_templates_no_cap)

    review = template.format(adjective=adjective, noun=noun, verb=verb)
    negative_reviews.append(review)

# Tạo danh sách các label
neg_labels = ["NEG"] * 10000

# Tạo danh sách các giá trị số sao ngẫu nhiên từ 1 đến 2
ratings = [random.randint(1, 2) for _ in range(10000)]

# Tạo DataFrame
df = pd.DataFrame({"content": negative_reviews, "label": neg_labels, "start": ratings})

# Lưu DataFrame vào file CSV
output_dir = "data"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_file = os.path.join(output_dir, "negative_reviews.csv")
df.to_csv(output_file, index=False, encoding="utf-8")

print(f"Đã lưu dữ liệu vào file: {output_file}")

Đã lưu dữ liệu vào file: data\negative_reviews.csv
